# Script to match the relevant tables for 4Hi-q

## Requirements

To be able to run this, the following requirements need to be met:

- A `python 3.10` environment, including the `astropy`, `astroquery` and standard modules
- The `function package` and the `catalogues` in the same directory.

## Structure

This script consists of a few separate parts to handle the catalogue data, divided into the following sections:

- Querying for the catalogue data and writing backups to disk, requiring the following:
  - A `region` specified in the beginning, used for RA and DEC constraints
  - An `agn_cat`, providing RA, DEC and probability information on the AGNs
  - The `sweep_cat` catalogues for the region in question (TODO: Use the LS DR query interface)
  - The `vhs_cat` is obtained by querying the Vista Science Archive
- Matching the catalogues, also writing a backup.
  - The matching is performed on the respective RA and DEC coordinates (TODO: Use the advanced constraints described in my thesis)
  - A match via the `cds` service is performed with the GALEX catalogue
- Processing the catalogues (unit conversions and reddening correction), including a split into *pointlike* and *extended*.
- Collapsing everything into a `LePhare` input file
- Running `LePhare`:
  - Collect the filter information
  - Collect the template information
  - Run the photo-z-routine

All of the functions used for these routines can either be found directly in this notebook or in the `function_package` directory.

In [17]:
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")

INFO:root:Modules loaded successfully


In [18]:
# Define the region to constrain everything to:
# The eFEDs field is dec < 6.2 & dec > -3.2 & ra < 146.2 & ra > 126.
ra_min, ra_max, dec_min, dec_max = 125, 130, -5, 0
REGION = fp.Region(ra_min, ra_max, dec_min, dec_max)
logging.info(REGION)

INFO:root:Region constrained to 125.0000 <= RA <= 130.0000 and -5.0000 <= DEC <= 0.0000.
This corresponds to a linear (!) size of 25 deg^2


## Loading the tables

In this section, the tables are loaded.\
TODO: Make use of astroquery if possible

In [21]:
SHU_TABLE = fp.load_and_clean_opt_agn_shu(REGION)

INFO:root:The reduced shu_agn table provided contains 2437 sources.
INFO:root:After the probability cut at p_rf >= 0.940, 1899 sources are left in the shu_agn table


In [15]:
VHS_TABLE = fp.load_and_clean_vhs(REGION)

INFO:root:The vhs table provided contains 1547498 sources.
INFO:root:The reduced vhs table provided contains 291905 sources.


In [16]:
SWEEP_TABLE = fp.load_and_clean_sweep(REGION)

INFO:root:The following bricks are in the requested region for the sweep table:
['120m005-130p000']
INFO:root:The reduced sweep table provided contains 2421599 sources.


## Matching

Now that the tables are properly loaded (except for GALEX, which we'll acquire while matching), we can match them one by one.

In [23]:
MATCH_TABLE = fp.match_with_sweep(SHU_TABLE, SWEEP_TABLE)
# TODO: Implement matching.
MATCH_TABLE = fp.match_with_vhs(MATCH_TABLE, VHS_TABLE)
MATCH_TABLE = fp.match_with_galex(MATCH_TABLE)

ModuleNotFoundError: No module named 'scipy'